# 1.清单计算

In [2]:
import csv,xlrd,xlwt,datetime,time,re
import numpy as np

import pandas as pd




#一、导入指导价

#二、打开对方清单
def suanqingdan():
    suanqingdan=0
    global url,qing,qing2,shijian0
    jia_df=pd.read_excel('转贴现买断价格计算/jia.xls',encoding='GB18030')

    shijian0=time.strftime('%Y-%m-%d',time.localtime(time.time()))
    shijian=datetime.datetime.strptime(shijian0 , "%Y-%m-%d")#转为日期格式

#2.1输入交易日
    jin=input('请输入要交易日,输入格式为“2017-11-01”，不输入默认为今天:')
    if jin!='':
        shijian=datetime.datetime.strptime(jin , "%Y-%m-%d")
    jia_df['生效截至日'] = pd.to_datetime(jia_df['生效截至日'])#.astype('str')   #,format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式 注意要换成string格式
    jia_df=jia_df[((jia_df['生效截至日']>=shijian) | (jia_df['生效截至日']==''))&(jia_df['银行分类']!='资管、证券、金融公司')]

    jia_df= jia_df.reset_index(drop=True)    #重新定义索引
    qixian1=jia_df.columns[2].split('至')
    qixian2=jia_df.columns[3].split('至')
    qixian3=jia_df.columns[4].split('至')
    qixian4=jia_df.columns[5].split('至')
    qixian5=jia_df.columns[6].split('至')

    qixian1[0] = datetime.datetime.strptime(qixian1[0] , "%Y-%m-%d")
    qixian1[1] = datetime.datetime.strptime(qixian1[1] , "%Y-%m-%d")
    qixian2[0] = datetime.datetime.strptime(qixian2[0] , "%Y-%m-%d")
    qixian2[1] = datetime.datetime.strptime(qixian2[1] , "%Y-%m-%d")
    qixian3[0] = datetime.datetime.strptime(qixian3[0] , "%Y-%m-%d")
    qixian3[1] = datetime.datetime.strptime(qixian3[1] , "%Y-%m-%d")
    qixian4[0] = datetime.datetime.strptime(qixian4[0] , "%Y-%m-%d")
    qixian4[1] = datetime.datetime.strptime(qixian4[1] , "%Y-%m-%d")
    qixian5[0] = datetime.datetime.strptime(qixian5[0] , "%Y-%m-%d")
    qixian5[1] = datetime.datetime.strptime(qixian5[1] , "%Y-%m-%d")


#2.2确认列名，到时候可以删除
    qing=pd.read_excel(url)
    print(qing.head())
#二、换列标题
    qingcolumn=qing.columns.values.tolist()  
    qingcolumn_df=pd.read_excel('转贴现买断价格计算/lieming.csv',encoding='GB18030')
    for i in range(0,len(qingcolumn)):
        for j in range(0,len(qingcolumn_df)):
            zhaop2= re.search(qingcolumn_df.loc[j,'原列名'] ,qingcolumn[i])
 

            if zhaop2:
                qing.rename(columns={qingcolumn[i]:qingcolumn_df.loc[j,'新列名']}, inplace=True)
                break

    if '贴现行' not in qing.columns.values.tolist():
        qing['贴现行']=''


#三、筛选列，计算清单
    qing=qing[['票号','票面金额','出票人','承兑行','贴现行','出票日','到期日']]
    qing= qing.reset_index(drop=True)    #重新定义索引
    for i in range(0,len(qing)):
        qing.loc[i,'承兑行号']=str(qing.loc[i,'票号'])[1:13]
        
        
    qing['到期日2'] = pd.to_datetime(qing['到期日'])   #把str转化为时间格式，format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式
    
    qing= qing.reset_index(drop=True)    #重新定义索引
    qing['承兑行号']=qing['承兑行号'].astype(str)
    
  #  qing['出票日']=qing['出票日'].astype(str)
    qing['出票日'] = pd.to_datetime(qing['出票日']).astype(str)   #把str转化为时间格式，format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式    
    qing['到期日'] = pd.to_datetime(qing['到期日']).astype(str)   #把str转化为时间格式，format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式    
        
  #  qing['到期日']=qing['到期日'].astype(str)

    for i in range (0,len(qing)):     
         if (qing.loc[i,'到期日2']>=qixian1[0]) and (qing.loc[i,'到期日2']<=qixian1[1]) :
           for j in range (0,len(jia_df)):
               if (jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行']) and (float(jia_df.loc[j,jia_df.columns[2]])>=1) :
                   if qing.loc[i,'票面金额']<1000000:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[2]]+0.2
                       qing.loc[i,'小票票据']=1
                   else:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[2]]
                       qing.loc[i,'小票票据']=0
                    

                   qing.loc[i,'指导期限']=jia_df.columns[2]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
               
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')

                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1

         elif (qing.loc[i,'到期日2']>=qixian2[0]) and (qing.loc[i,'到期日2']<=qixian2[1]) :

           for j in range (0,len(jia_df)):

               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行']    and (float(jia_df.loc[j,jia_df.columns[3]])>=1) :
                   if qing.loc[i,'票面金额']<1000000:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[3]]+0.2
                       qing.loc[i,'小票票据']=1
                   else:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[3]]
                       qing.loc[i,'小票票据']=0
                    
                   qing.loc[i,'指导期限']=jia_df.columns[3]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]
                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1

         elif (qing.loc[i,'到期日2']>=qixian3[0]) and (qing.loc[i,'到期日2']<=qixian3[1])  :  #

           for j in range (0,len(jia_df)):
               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行']  and (float(jia_df.loc[j,jia_df.columns[4]])>=1):
                   if qing.loc[i,'票面金额']<1000000:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[4]]+0.2
                       qing.loc[i,'小票票据']=1
                   else:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[4]]
                       qing.loc[i,'小票票据']=0

                   qing.loc[i,'指导期限']=jia_df.columns[4]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
               
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1

         elif (qing.loc[i,'到期日2']>=qixian4[0]) and (qing.loc[i,'到期日2']<=qixian4[1])  :

           for j in range (0,len(jia_df)) :
               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行'] and (float(jia_df.loc[j,jia_df.columns[5]])>=1):
                   if qing.loc[i,'票面金额']<1000000:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[5]]+0.15
                       qing.loc[i,'小票票据']=1
                   else:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[5]]
                       qing.loc[i,'小票票据']=0
                   qing.loc[i,'指导期限']=jia_df.columns[5]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
               
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1




         elif (qing.loc[i,'到期日2']>=qixian5[0]) and (qing.loc[i,'到期日2']<=qixian5[1])  :

           for j in range (0,len(jia_df)) :
               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行'] and (float(jia_df.loc[j,jia_df.columns[6]])>=1):
                   if qing.loc[i,'票面金额']<1000000:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[6]]+0.15
                       qing.loc[i,'小票票据']=1
                   else:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[6]]
                       qing.loc[i,'小票票据']=0
                   qing.loc[i,'指导期限']=jia_df.columns[6]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
               
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1
                   
    if suanqingdan<=0:
        print('整个清单都不能收！或者请清单检查格式。提示：1.出票日、到期日要是日期格式，2.是否有其他“承兑”、“付款”之类的列名存在，系统误认为是错误的“承兑行”列。')
        del qing['到期日2']
        qing2=qing
        
    else:
        qing['加权利率']=qing.loc[:,'ji2'].sum()/qing.loc[:,'ji1'].sum()
        del qing['到期日2']



        qing=qing[['票号','票面金额','出票人','承兑行','承兑行号','出票日','到期日','交易日','银行分类','邮储指导价','小票票据','指导期限','加权利率','贴现行','可用额度','我行评级','生效截至日','授信有效','期限','到期月']]
#三、将算好的清单写入excel
        qing2=qing.replace(np.nan,99)
     
        qing2=qing2[qing2['邮储指导价'] != 99]  #排除NAN
        chengdui=qing2['票面金额'].groupby(qing2['银行分类']).sum()/10000
        print('  ')

        zongjin=qing2['票面金额'].sum()/10000
        print('总金额：%s'%zongjin)
        print('  清单分类如下：')

        print(chengdui)
        print('  ')
        yuefen=qing2['票面金额'].groupby(qing2['到期月']).sum()/10000
        print(yuefen)
        print('  ')
        qixian=qing2['期限'].mean()
        print(qixian)
        print('  ')
        print('提示：票据持有方式选：持有！申报清单总金额和张数是否和对方实际发来的一致？承兑行是否有错别字？')

    
    





#四、制作导
        qing2['导汇票']=1
        qing2['导出票人账号']=''
        qing2['导收款人名称']=''
        qing2['导1']=''
        qing2['导2']=''
        qing2['导3']=''
        qing2['同城']=1
        qing2=qing2[['导汇票','票号','票面金额','出票日','到期日','出票人','导出票人账号','承兑行号','承兑行','导收款人名称','导1','导2','导3','承兑行号','同城','交易日','期限']]

        qing2=qing2.fillna('')  #把日期列的nan 数值用空值填充，统一空值


        qing2=qing2[(qing2['票号'] !='')]
        qing2= qing2.reset_index(drop=True)    #重新定义索引
        qing=qing.fillna('')  #把日期列的nan 数值用“NAN”填充，统一空值


        qing=qing[(qing['票号'] !='')]
        qing= qing.reset_index(drop=True)    #重新定义索引
        
        
def heimingdan():
    global qing,qing2,shijian0
    hei1=pd.read_excel('转贴现买断价格计算/系统黑名单.xls',encoding='GB18030')
  #  hei2=pd.read_excel('转贴现买断价格计算/系统黑名单企业.xls',encoding='GB18030')           
    
    for i in range(0,len(hei1)):
        if '/' in  hei1.loc[i,'银行黑名单']:      
            erhei= hei1.loc[i,'银行黑名单'].split('/')
            for j in range(0,len(qing)):
                if (erhei[0] in qing.loc[j,'承兑行']) and (erhei[1] in qing.loc[j,'承兑行']):
                    qing.loc[j,'黑名单']=hei1.loc[i,'黑名单类型']
                    qing2.loc[j,'黑名单']=hei1.loc[i,'黑名单类型']
            
                   # break
            
            
            
        else:
            for j in range(0,len(qing)):
                if hei1.loc[i,'银行黑名单'] in qing.loc[j,'承兑行']:
                    qing.loc[j,'黑名单']=hei1.loc[i,'黑名单类型']
                    qing2.loc[j,'黑名单']=hei1.loc[i,'黑名单类型']
                  #  break
    url3='临时清单/'+shijian0+duifang+'导入.xls'
    writer = pd.ExcelWriter(url3)
    qing2.to_excel(writer,'Sheet1')
    writer.save()

    url2='临时清单/'+shijian0+duifang+'算加权.xls'
    writer = pd.ExcelWriter(url2)
    qing.to_excel(writer,'Sheet1')
    writer.save()

        
for i in range(0,1):
    print('\n')
    duifang=input('请输入第%s个文件名：'%i)
 #   duifang=''+duifang
    url=('临时清单/%s'%duifang)
    if duifang=='':
        continue
    else:
        
        suanqingdan()
        heimingdan() 




请输入第0个文件名：跨年城商.xls
请输入要交易日,输入格式为“2017-11-01”，不输入默认为今天:
   序号                            汇票号码         出票日       票面到期日        金额  \
0   1  131358401802720190430389883860  2019-04-30  2020-01-22  850000.0   
1   2  131346889422220190125338381164  2019-01-25  2020-01-25  700000.0   
2   3  131379100092920190201346835739  2019-02-01  2020-01-30  700000.0   
3   4  131358109405420190130345107190  2019-01-30  2020-01-30  684000.0   
4   5  131465300003820190131345721276  2019-01-31  2020-01-31  600000.0   

            出票人                   承兑行  
0  上海盾金物资贸易有限公司  珠海华润银行股份有限公司深圳分行清算中心  
1  禹城宝泰机械制造有限公司     威海市商业银行股份有限公司德州分行  
2  陕西金汤城建工程有限公司              西安银行高新支行  
3  广州天力建筑工程有限公司      九江银行股份有限公司广州环市支行  
4  中铁隧道集团一处有限公司    重庆农村商业银行股份有限公司江北支行  
  
总金额：13175.351425
  清单分类如下：
银行分类
二类准入、外资     2599.685361
其他授信银行     10575.666064
Name: 票面金额, dtype: float64
  
到期月
2020-02      228.286679
2020-03      301.685361
2020-04      357.000000
2020-05      947.213750
2020-06    11341.165635
Name: 票面金额, dty

In [1]:
# -*- coding: cp936 -*-
import datetime
import json,csv,re,time
import pandas as pd
from pymongo import MongoClient


client=MongoClient('mongodb://root:' + '5768116' + '@121.196.220.14')
db = client.cundan
collection = db.cundan





shijian=time.strftime('%Y-%m-%d',time.localtime(time.time()))
shijian=str(shijian)
shijian = datetime.datetime.strptime(shijian, "%Y-%m-%d")
shijian1=shijian-datetime.timedelta(days=20)




url='yinhang.csv'
df = pd.read_table(url,sep=',',encoding='GB18030')   #用dictreader根据行内容查找


for i in range(0,len(df)):    
        cursor = collection.find({"$and":[{'发行日':{'$gte':str(shijian1)}},{'发行人':{'$regex':df.loc[i,'银行']}}]}) 
        cun_df = pd.DataFrame(list(cursor))
        if cun_df.empty:

            print('11')
        else:
            cun_df1=cun_df[cun_df['期限']=='1月']
            cun_df1=cun_df1.reset_index(drop=False)

            if cun_df1.empty:
               print(111)
            else:
                df.loc[i,'1月利率']=cun_df1.loc[(len(cun_df1)-1),'收益率']
                df.loc[i,'1月发行日']=cun_df1.loc[(len(cun_df1)-1),'发行日']
                df.loc[i,'1月发行量']=cun_df1.loc[(len(cun_df1)-1),'计划发行(亿)']

            cun_df2=cun_df[cun_df['期限']=='3月']
            cun_df2=cun_df2.reset_index(drop=False)

            if cun_df2.empty:
               print(111)
            else:
                print(cun_df2)
                df.loc[i,'3月利率']=cun_df2.loc[(len(cun_df2)-1),'收益率']
                df.loc[i,'3月发行日']=cun_df2.loc[(len(cun_df2)-1),'发行日']
                df.loc[i,'3月发行量']=cun_df2.loc[(len(cun_df2)-1),'计划发行(亿)']
                
            cun_df3=cun_df[cun_df['期限']=='6月']
            cun_df3=cun_df3.reset_index(drop=False)

            if cun_df3.empty:
               print(111)
            else:
                df.loc[i,'6月利率']=cun_df3.loc[(len(cun_df3)-1),'收益率']
                df.loc[i,'6月发行日']=cun_df3.loc[(len(cun_df3)-1),'发行日']
                df.loc[i,'6月发行量']=cun_df3.loc[(len(cun_df3)-1),'计划发行(亿)']

            cun_df4=cun_df[cun_df['期限']=='9月']
            cun_df4=cun_df4.reset_index(drop=False)

            if cun_df4.empty:
               print(111)
            else:
                df.loc[i,'9月利率']=cun_df4.loc[(len(cun_df4)-1),'收益率']
                df.loc[i,'9月发行日']=cun_df4.loc[(len(cun_df4)-1),'发行日']
                df.loc[i,'9月发行量']=cun_df4.loc[(len(cun_df4)-1),'计划发行(亿)']



            cun_df5=cun_df[cun_df['期限']=='1年']
            cun_df5=cun_df5.reset_index(drop=False)

            if cun_df5.empty:
               print(111)
            else:
                df.loc[i,'1年利率']=cun_df5.loc[(len(cun_df5)-1),'收益率']
                df.loc[i,'1年发行日']=cun_df5.loc[(len(cun_df5)-1),'发行日']
                df.loc[i,'1年发行量']=cun_df5.loc[(len(cun_df5)-1),'计划发行(亿)']



#cursor = collection.find({ "$and":[{'发行日':'2018-03-06'}] })  

#df = pd.DataFrame(list(cursor))

#print(df)
df.to_csv('重要交易对手同业存单.csv',mode='w',header=True)


11
11
11
11
111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        2  5d03919612e3441ef87acd85      3  111904039  19中国银行CD039  报价发行   

          发行日 实际发行(亿) 息票类型  收益率  期限        爬取日期 票面利率  计划发行(亿)   评级  银行分类  
0  2019-06-17    None   零息  2.9  3月  2019-06-14   --     30.0  AAA  中国银行  
111
111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        1  5c45928fe5b12d1ee0a53553    102  111906015  19交通银行CD015  报价发行   
1        5  5cf8f88c12e344151808e4be      3  111906158  19交通银行CD158  报价发行   
2        8  5cfe49f112e3441824a0c2db      5  111906161  19交通银行CD161  报价发行   
3       12  5d00db5a12e344084c8ab6d5      3  111906165  19交通银行CD165  报价发行   
4       17  5d03919612e3441ef87acd89      7  111906170  19交通银行CD170  报价发行   
5       19  5d076ea312e34412c8e68ea7      5  111906172  19交通银行CD172  报价发行   

          发行日  实际发行(亿) 息票类型   收益率  期限        爬取日期 票面利率  计划发行(亿)   评级  银行分类  
0  2019/01/21      8.2   零息  2.79  3月  2019-01-21   --

    level_0                       _id  index       债券代码          发行人  发行方式  \
0         1  5cf8f88c12e344151808e4d8     29  111921190  19渤海银行CD190  报价发行   
1         3  5cfe49f112e3441824a0c302     44  111921192  19渤海银行CD192  报价发行   
2         5  5cff9e9112e344084c8ab151     29  111921194  19渤海银行CD194  报价发行   
3         8  5d00db5a12e344084c8ab6e9     23  111921197  19渤海银行CD197  报价发行   
4        10  5d02358f12e344178411cb0d     22  111921199  19渤海银行CD199  报价发行   
5        12  5d03919612e3441ef87acd9b     25  111921201  19渤海银行CD201  报价发行   
6        15  5d076ea312e34412c8e68ed1     47  111921204  19渤海银行CD204  报价发行   
7        17  5d076ea312e34412c8e68ed3     49  111921206  19渤海银行CD206  报价发行   
8        19  5d08bfc812e3441e6493160d     31  111921208  19渤海银行CD208  报价发行   
9        20  5d08bfc812e3441e6493160e     32  111921209  19渤海银行CD209  报价发行   
10       22  5d08bfc812e3441e64931610     34  111921211  19渤海银行CD211  报价发行   
11       24  5d0a373012e34421707d56c9     27  111921213  19渤海银行C

111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        0  5cfe49f112e3441824a0c39a    196  111999714  19赣州银行CD067  报价发行   
1        1  5cff9e9112e344084c8ab1eb    183  111999871  19赣州银行CD068  报价发行   
2        2  5d00db5a12e344084c8ab75d    139  111999982  19赣州银行CD069  报价发行   
3        3  5d02358f12e344178411cb6d    118  111980099  19赣州银行CD070  报价发行   
4        4  5d03919612e3441ef87acddf     93  111980236  19赣州银行CD071  报价发行   
5        5  5d076ea312e34412c8e68efc     90  111980355  19赣州银行CD072  报价发行   
6        6  5d08bfc812e3441e64931636     72  111980540  19赣州银行CD073  报价发行   
7        7  5d0a373012e34421707d56fb     77  111980698  19赣州银行CD074  报价发行   
8        8  5d0b8aa912e3440a200cd183    161  111980926  19赣州银行CD075  报价发行   
9        9  5d11fbe012e3441ad822a368    132  111981324  19赣州银行CD076  报价发行   

          发行日 实际发行(亿) 息票类型  收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-06-11    None   零息  3.1  3月  2019-06-10   --      5.0  AA+   城商  


   level_0                       _id  index       债券代码          发行人  发行方式  \
0        4  5d02358f12e344178411cba3    172  111980154  19晋商银行CD060  报价发行   
1        5  5d03919612e3441ef87acdfe    124  111980267  19晋商银行CD061  报价发行   
2        7  5d076ea312e34412c8e68f68    198  111980463  19晋商银行CD063  报价发行   
3       10  5d0a373012e34421707d573f    145  111980767  19晋商银行CD066  报价发行   

          发行日 实际发行(亿) 息票类型   收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-06-14    None   零息  3.35  3月  2019-06-13   --      4.0  AA+   城商  
1  2019-06-17    None   零息  3.35  3月  2019-06-14   --      0.7  AA+   城商  
2  2019-06-18    None   零息  3.30  3月  2019-06-17   --      2.0  AA+   城商  
3  2019-06-20    None   零息  3.30  3月  2019-06-19   --      4.2  AA+   城商  
111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        0  5d0cf10412e3441600ad51e3    120  111981057  19晋中银行CD006  报价发行   

          发行日 实际发行(亿) 息票类型   收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  20

    level_0                       _id  index       债券代码          发行人  发行方式  \
0         1  5cf8f88c12e344151808e4d2     23  111916142  19上海银行CD142  报价发行   
1         4  5cfe49f112e3441824a0c2f9     35  111916145  19上海银行CD145  报价发行   
2         5  5cfe49f112e3441824a0c2fa     36  111916146  19上海银行CD146  报价发行   
3         8  5cff9e9112e344084c8ab148     20  111916149  19上海银行CD149  报价发行   
4        10  5d00db5a12e344084c8ab6e5     19  111916151  19上海银行CD151  报价发行   
5        12  5d02358f12e344178411cb0b     20  111916153  19上海银行CD153  报价发行   
6        14  5d03919612e3441ef87acd99     23  111916155  19上海银行CD155  报价发行   
7        16  5d076ea312e34412c8e68ec3     33  111916157  19上海银行CD157  报价发行   
8        18  5d08bfc812e3441e64931607     25  111916159  19上海银行CD159  报价发行   
9        20  5d0a373012e34421707d56c1     19  111916161  19上海银行CD161  报价发行   
10       21  5d0a373012e34421707d56c2     20  111916162  19上海银行CD162  报价发行   
11       23  5d0b8aa912e3440a200cd0fa     24  111916164  19上海银行C

8  2019-06-26    None   零息  2.65  3月  2019-06-25   --      0.8  AAA   城商  
111
11
111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        3  5d11fbe012e3441ad822a359    117  111981309  19潍坊银行CD046  报价发行   

          发行日 实际发行(亿) 息票类型  收益率  期限        爬取日期 票面利率  计划发行(亿)  评级 银行分类  
0  2019-06-26    None   零息  3.5  3月  2019-06-25   --      0.5  AA   城商  
111
111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        3  5d03919612e3441ef87acdf7    117  111980260  19温州银行CD056  报价发行   
1        5  5d076ea312e34412c8e68f2b    137  111980402  19温州银行CD058  报价发行   
2        7  5d08bfc812e3441e64931653    101  111980569  19温州银行CD060  报价发行   
3       10  5d0a373012e34421707d5714    102  111980724  19温州银行CD063  报价发行   

          发行日 实际发行(亿) 息票类型   收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-06-17    None   零息  3.41  3月  2019-06-14   --      1.0  AA+   城商  
1  2019-06-18    None   零息  3.32  3月  2019-06-17   --      2.5  AA+  

111
111
111
111
11
   level_0                       _id  index       债券代码                发行人  \
0        5  5d076ea312e34412c8e68ee7     69  111980334  19江苏江南农村商业银行CD060   

   发行方式         发行日 实际发行(亿) 息票类型   收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  报价发行  2019-06-18    None   零息  3.03  3月  2019-06-17   --      5.0  AAA  农商行  
11
11
11
   level_0                       _id  index       债券代码             发行人  发行方式  \
0        0  5cf8f88c12e344151808e4e3     40  111999467  19江苏紫金农商行CD044  报价发行   
1        5  5d076ea312e34412c8e68ee6     68  111980333  19江苏紫金农商行CD049  报价发行   

          发行日 实际发行(亿) 息票类型   收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-06-10    None   零息  3.25  3月  2019-06-06   --      5.0  AA+  农商行  
1  2019-06-18    None   零息  3.33  3月  2019-06-17   --      5.0  AA+  农商行  
   level_0                       _id  index       债券代码           发行人  发行方式  \
0        1  5cfe49f112e3441824a0c381    171  111999689  19厦门农商行CD014  报价发行   
1       11  5d03919612e3441ef87ac

11
11
111
   level_0                       _id  index       债券代码            发行人  发行方式  \
0        0  5cf8f88c12e344151808e4fa     63  111999490  19重庆农村商行CD123  报价发行   
1        3  5cfe49f112e3441824a0c354    126  111999644  19重庆农村商行CD126  报价发行   
2        7  5cff9e9112e344084c8ab1a6    114  111999801  19重庆农村商行CD130  报价发行   
3       10  5d00db5a12e344084c8ab734     98  111999941  19重庆农村商行CD133  报价发行   
4       13  5d02358f12e344178411cb68    113  111980094  19重庆农村商行CD136  报价发行   
5       19  5d076ea312e34412c8e68f2d    139  111980404  19重庆农村商行CD142  报价发行   
6       21  5d0cf10412e3441600ad51bc     81  111981018  19重庆农村商行CD144  报价发行   
7       22  5d10c0a212e3441d646c653d     52  111981127  19重庆农村商行CD145  报价发行   
8       23  5d11fbe012e3441ad822a33b     87  111981278  19重庆农村商行CD146  报价发行   

          发行日 实际发行(亿) 息票类型   收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-06-10    None   零息  3.08  3月  2019-06-06   --      5.0  AAA  农商行  
1  2019-06-11    None   零息  3.10  3月  2019-06-10 

#  微信群价格

In [2]:
# -*- coding: cp936 -*-
import pandas as pd

import matplotlib.pyplot as plt
#from PIL import Image
import numpy as np

import jieba,os
import jieba.analyse
import json,csv,re,time,datetime
from pymongo import MongoClient
#from wordcloud import WordCloud
shijian=time.strftime('%Y-%m-%d',time.localtime(time.time()))

shijian = datetime.datetime.strptime(shijian, "%Y-%m-%d")
shijian=str(shijian.strftime("%Y-%m-%d"))

print(shijian)
shuru=input('请输入要统计的日期,格式为“2017-11-01”，不输入自定义为今天:')
if shuru!='':
    shijian=shuru

client=MongoClient('mongodb://root:' + '5768116' + '@121.196.220.14')
db = client.piaofen
collection = db.piaofen
cursor = collection.find({"$and":[{'time':shijian},{'hanglei2':1}]})
df = pd.DataFrame(list(cursor))

df = df.sort_values(by='time2', ascending=True)
df=df.drop_duplicates(['content'],keep='last')   #删除连接中的重复行

for i in range (1,len(df)):
      zhaop= re.search('[^0-9][0-9]\.[0-9]+[^元万亿]',str(df.loc[i,'content']))
      if zhaop:
         df.loc[i,'jiage']=1
      zhaop1= re.search('[0-9]+%',str(df.loc[i,'content']))
      if zhaop1:
         df.loc[i,'jiage']=1

for i in range (1,len(df)):   
      zhaop3= re.search('[^0-9][1-9][0-9][0-9][^0-9万天]',str(df.loc[i,'content']))
      if zhaop3:
             df.loc[i,'jiage']=1
      

df=df[df['jiage']==1]
df1=df[(df['shou']+df['chu']+df['shoudai']+df['chudai']+df['shouhui']+df['chuhui'])>0]
df1=df1.sort_values('hanglei1', ascending=True)  #吧银行类别排序

columns = ['time','time2','hanglei1','nickname','content']
df1.to_csv('票据价格.csv',mode='w',header=True,columns=columns)

df2=df[(df['shoufu']+df['chufu'])>0]
df2=df2.sort_values('hanglei1', ascending=True)  #吧银行类别排序
columns = ['time','time2','hanglei1','nickname','content']
df2.to_csv('福费廷价格.csv',mode='w',header=True,columns=columns)

df3=df[(df['shoucun']+df['chucun'])>0]
df3=df3.sort_values('hanglei1', ascending=True)  #吧银行类别排序
columns = ['time','time2','hanglei1','nickname','content']
df3.to_csv('资金价格.csv',mode='w',header=True,columns=columns)

df4=df[(df['shouli']+df['chuli'])>0]
df4=df4.sort_values('hanglei1', ascending=True)  #吧银行类别排序
columns = ['time','time2','hanglei1','nickname','content']
df4.to_csv('理财价格.csv',mode='w',header=True,columns=columns)


2019-06-27
请输入要统计的日期,格式为“2017-11-01”，不输入自定义为今天:
